In [2]:
! git clone https://github.com/cyizhuo/CUB-200-2011-dataset.git

Cloning into 'CUB-200-2011-dataset'...
remote: Enumerating objects: 12221, done.
remote: Total 12221 (delta 0), reused 0 (delta 0), pack-reused 12221
Receiving objects: 100% (12221/12221), 1.05 GiB | 26.43 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (11789/11789), done.


In [3]:
# Import necessary libraries
import tensorflow as tf

# Load the pre-trained Inception-ResNet V2 model without the top layers
model = tf.keras.applications.InceptionResNetV2(include_top=False, weights='imagenet')

# Print the model summary
model.summary()


Model: "inception_resnet_v2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv2d_203 (Conv2D)         (None, None, None, 32)       864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_203 (B  (None, None, None, 32)       96        ['conv2d_203[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_203 (Activation  (None, None, None, 32)       0         ['batch_norm

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Check if GPU is available
if tf.config.experimental.list_physical_devices('GPU'):
    print('GPU is available. Using CUDA.')
else:
    print('GPU is not available. Using CPU.')

# Load the pre-trained Inception-ResNet V2 model without the top layers
base_model = tf.keras.applications.InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Modify the model to suit the number of classes in the CUB dataset
num_classes = 200  # CUB dataset has 200 classes

# Add custom classification head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Print model summary
model.summary()

# Load CUB dataset
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/CUB-200-2011-dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/CUB-200-2011-dataset/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# Optionally, you can fine-tune the entire model by unfreezing the base model layers and recompiling the model.
# base_model.trainable = True
# model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])
# history = model.fit(...)


GPU is available. Using CUDA.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Funct  (None, 5, 5, 1536)        54336736  
 ional)                                                          
                                                                 
 global_average_pooling2d_1  (None, 1536)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_2 (Dense)             (None, 1024)              1573888   
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 200)               205000    
                                                                 
Total params: 56115624 (

KeyboardInterrupt: 

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Load the pre-trained Inception model and freeze its layers
inception_model = models.inception_v3(pretrained=True)
for param in inception_model.parameters():
    param.requires_grad = False

# Modify the inception model's output to match the number of classes in CUB dataset
inception_model.fc = nn.Linear(inception_model.fc.in_features, 200)

# Load the pre-trained ResNet-18 model
resnet_model = models.resnet50(pretrained=True)

# Modify the ResNet-18 model's output to match the number of classes in CUB dataset
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 200)

train_dir = '/content/CUB-200-2011-dataset/train'
val_dir = '/content/CUB-200-2011-dataset/test'

# Define transformations for data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

# Load the CUB dataset
# Load the CUB dataset
image_datasets = {x: datasets.ImageFolder(root=train_dir if x == 'train' else val_dir, transform=data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2) for x in ['train', 'val']}

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)

# Train the ResNet-18 model
resnet_model.to(device)
resnet_model.train()
for epoch in range(10):  # Adjust the number of epochs as needed
    running_loss = 0.0
    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(dataloaders['train'].dataset)
    print(f"Epoch {epoch+1}/{10}, Loss: {epoch_loss:.4f}")

# Optionally, evaluate the model on the validation set




Using device: cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-d

Epoch 1/10, Loss: 5.3700
Epoch 2/10, Loss: 5.0362
Epoch 3/10, Loss: 4.7875
Epoch 4/10, Loss: 4.5910
Epoch 5/10, Loss: 4.4562
Epoch 6/10, Loss: 4.2665
Epoch 7/10, Loss: 4.1266
Epoch 8/10, Loss: 3.9432
Epoch 9/10, Loss: 3.8123
Epoch 10/10, Loss: 3.6841


In [6]:
correct = 0
total = 0
resnet_model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient computation
    for images, labels in dataloaders['val']:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet_model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the validation set: {100 * correct / total:.2f}%')

Accuracy on the validation set: 16.59%


## Trial 2 VGG

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Load the pre-trained VGG16 model and freeze its layers
vgg_model = models.vgg16(pretrained=True)
for param in vgg_model.parameters():
    param.requires_grad = False

# Modify the VGG16 model's output to match the number of classes in CUB dataset
num_classes = 200  # Number of classes in CUB dataset
vgg_model.classifier[-1] = nn.Linear(vgg_model.classifier[-1].in_features, num_classes)

train_dir = '/content/CUB-200-2011-dataset/train'
val_dir = '/content/CUB-200-2011-dataset/test'

# Define transformations for data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

# Load the CUB dataset
image_datasets = {x: datasets.ImageFolder(root=train_dir if x == 'train' else val_dir, transform=data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2) for x in ['train', 'val']}

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg_model.parameters(), lr=0.001)

# Train the VGG16 model
vgg_model.to(device)
vgg_model.train()
for epoch in range(10):  # Adjust the number of epochs as needed
    running_loss = 0.0
    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = vgg_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(dataloaders['train'].dataset)
    print(f"Epoch {epoch+1}/{10}, Loss: {epoch_loss:.4f}")

# Optionally, evaluate the model on the validation set
correct = 0
total = 0
vgg_model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient computation
    for images, labels in dataloaders['val']:
        images, labels = images.to(device), labels.to(device)
        outputs = vgg_model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the validation set: {100 * correct / total:.2f}%')


Using device: cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 75.7MB/s]


Epoch 1/10, Loss: 3.4894
Epoch 2/10, Loss: 2.3955
Epoch 3/10, Loss: 2.1588
Epoch 4/10, Loss: 2.1331
Epoch 5/10, Loss: 2.0151
Epoch 6/10, Loss: 1.9804
Epoch 7/10, Loss: 2.0138
Epoch 8/10, Loss: 1.9433
Epoch 9/10, Loss: 1.9762
Epoch 10/10, Loss: 1.9616
Accuracy on the validation set: 59.48%


In [8]:
# Calculate the number of total parameters in the model
total_params = sum(p.numel() for p in vgg_model.parameters())
print(f'Total parameters in the model: {total_params}')

# Calculate the number of trainable parameters in the model
trainable_params = sum(p.numel() for p in vgg_model.parameters() if p.requires_grad)
print(f'Total trainable parameters in the model: {trainable_params}')


Total parameters in the model: 135079944
Total trainable parameters in the model: 819400


## Trial 3
FineTuning on Unfreezing Some layers

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dir = '/content/CUB-200-2011-dataset/train'
val_dir = '/content/CUB-200-2011-dataset/test'

# Define transformations for data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

# Load the CUB dataset
image_datasets = {x: datasets.ImageFolder(root=train_dir if x == 'train' else val_dir, transform=data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2) for x in ['train', 'val']}

# Load pre-trained VGG16
vgg16 = models.vgg16(pretrained=True)

# Freeze all layers
for param in vgg16.parameters():
    param.requires_grad = False

# Unfreeze some convolutional layers
for i in range(10):
    for param in vgg16.features[-i].parameters():
        param.requires_grad = True

# Modify the classifier
num_features = vgg16.classifier[6].in_features
vgg16.classifier[6] = nn.Linear(num_features, 200)

# Move model to device
vgg16 = vgg16.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    vgg16.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in dataloaders['train']:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(dataloaders['train'].dataset)
    train_accuracy = correct_train / total_train

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Accuracy: {train_accuracy:.2%}")

    # Evaluation loop
    vgg16.eval()
    correct_val = 0
    total_val = 0
    val_running_loss = 0.0

    with torch.no_grad():
        for images, labels in dataloaders['val']:
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            val_loss = criterion(outputs, labels)
            val_running_loss += val_loss.item() * images.size(0)

    val_epoch_loss = val_running_loss / len(dataloaders['val'].dataset)
    val_accuracy = correct_val / total_val

    print(f"Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}")

# Print final test accuracy
print(f"Final Test Accuracy: {val_accuracy:.2%}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/20], Loss: 4.4325
Epoch [2/20], Loss: 2.7754
Epoch [3/20], Loss: 2.2339
Epoch [4/20], Loss: 1.9940
Epoch [5/20], Loss: 1.7927
Epoch [6/20], Loss: 1.6844
Epoch [7/20], Loss: 1.5739
Epoch [8/20], Loss: 1.4961
Epoch [9/20], Loss: 1.4105
Epoch [10/20], Loss: 1.3621
Epoch [11/20], Loss: 1.3106
Epoch [12/20], Loss: 1.2844
Epoch [13/20], Loss: 1.2286
Epoch [14/20], Loss: 1.2040
Epoch [15/20], Loss: 1.1629
Epoch [16/20], Loss: 1.0885
Epoch [17/20], Loss: 1.0909
Epoch [18/20], Loss: 1.0458
Epoch [19/20], Loss: 0.9779
Epoch [20/20], Loss: 0.9909
Accuracy on validation set: 72.51%


In [18]:
# Calculate the number of total parameters in the model
total_params = sum(p.numel() for p in vgg16.parameters())
print(f'Total parameters in the model: {total_params}')

# Calculate the number of trainable parameters in the model
trainable_params = sum(p.numel() for p in vgg16.parameters() if p.requires_grad)
print(f'Total trainable parameters in the model: {trainable_params}')


Total parameters in the model: 135079944
Total trainable parameters in the model: 7900616


## Trial 4

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dir = '/content/CUB-200-2011-dataset/train'
val_dir = '/content/CUB-200-2011-dataset/test'

# Define transformations for data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

# Load the CUB dataset
image_datasets = {x: datasets.ImageFolder(root=train_dir if x == 'train' else val_dir, transform=data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2) for x in ['train', 'val']}

# Load pre-trained VGG16
vgg16 = models.vgg16(pretrained=True)

# Freeze all layers
for param in vgg16.parameters():
    param.requires_grad = False

# Unfreeze some convolutional layers
for i, param in enumerate(vgg16.features.parameters(), 1):
    if i > 15:  # Adjust the number of layers to unfreeze as needed
        param.requires_grad = True

# Modify the classifier
num_features = vgg16.classifier[6].in_features
vgg16.classifier[6] = nn.Linear(num_features, 200)  # Set to 200 for CUB dataset


# Move model to device
vgg16 = vgg16.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 25
for epoch in range(num_epochs):
    vgg16.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in dataloaders['train']:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(dataloaders['train'].dataset)
    train_accuracy = correct_train / total_train

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Accuracy: {train_accuracy:.2%}")

    # Evaluation loop
    vgg16.eval()
    correct_val = 0
    total_val = 0
    val_running_loss = 0.0

    with torch.no_grad():
        for images, labels in dataloaders['val']:
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            val_loss = criterion(outputs, labels)
            val_running_loss += val_loss.item() * images.size(0)

    val_epoch_loss = val_running_loss / len(dataloaders['val'].dataset)
    val_accuracy = correct_val / total_val

    print(f"Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}")

# Print final test accuracy
print(f"Final Test Accuracy: {val_accuracy:.2%}")


Epoch [1/25], Train Loss: 4.3873, Train Accuracy: 10.33%
Validation Loss: 2.4603, Validation Accuracy: 37.50%
Epoch [2/25], Train Loss: 2.6973, Train Accuracy: 33.12%
Validation Loss: 1.6088, Validation Accuracy: 55.83%
Epoch [3/25], Train Loss: 2.1916, Train Accuracy: 43.88%
Validation Loss: 1.4274, Validation Accuracy: 59.42%
Epoch [4/25], Train Loss: 1.8801, Train Accuracy: 51.13%
Validation Loss: 1.2382, Validation Accuracy: 65.50%
Epoch [5/25], Train Loss: 1.7157, Train Accuracy: 53.85%
Validation Loss: 1.1386, Validation Accuracy: 68.45%
Epoch [6/25], Train Loss: 1.5927, Train Accuracy: 57.66%
Validation Loss: 1.1649, Validation Accuracy: 67.26%
Epoch [7/25], Train Loss: 1.4792, Train Accuracy: 60.68%
Validation Loss: 1.0694, Validation Accuracy: 70.11%
Epoch [8/25], Train Loss: 1.4144, Train Accuracy: 62.88%
Validation Loss: 1.0397, Validation Accuracy: 70.95%
Epoch [9/25], Train Loss: 1.3412, Train Accuracy: 64.30%
Validation Loss: 1.0389, Validation Accuracy: 70.62%
Epoch [10/

## Trial 5
FineTuning the Layers of VGG 19

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

train_dir = '/content/CUB-200-2011-dataset/train'
val_dir = '/content/CUB-200-2011-dataset/test'

# Define transformations for data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

# Load the CUB dataset
image_datasets = {x: datasets.ImageFolder(root=train_dir if x == 'train' else val_dir, transform=data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2) for x in ['train', 'val']}

# Load pre-trained VGG19
vgg19 = models.vgg19(weights='DEFAULT')

# Freeze all layers
for param in vgg19.parameters():
    param.requires_grad = False

# Unfreeze some convolutional layers
for i, param in enumerate(vgg19.features.parameters(), 1):
    if i > 20:  # Adjust the number of layers to unfreeze as needed
        param.requires_grad = True

# Modify the classifier
num_features = vgg19.classifier[6].in_features
vgg19.classifier[6] = nn.Linear(num_features, 200)  # Set to 200 for CUB dataset

# Move model to device
vgg19 = vgg19.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg19.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 25
for epoch in range(num_epochs):
    vgg19.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in dataloaders['train']:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg19(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(dataloaders['train'].dataset)
    train_accuracy = correct_train / total_train

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Accuracy: {train_accuracy:.2%}")

    # Evaluation loop
    vgg19.eval()
    correct_val = 0
    total_val = 0
    val_running_loss = 0.0

    with torch.no_grad():
        for images, labels in dataloaders['val']:
            images, labels = images.to(device), labels.to(device)
            outputs = vgg19(images)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            val_loss = criterion(outputs, labels)
            val_running_loss += val_loss.item() * images.size(0)

    val_epoch_loss = val_running_loss / len(dataloaders['val'].dataset)
    val_accuracy = correct_val / total_val

    print(f"Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}")

# Print final test accuracy
print(f"Final Test Accuracy: {val_accuracy:.2%}")


## Trial 6
Efficient Net

In [1]:
!pip install efficientnet-pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=740faa7ebbcf35b5a07f85fd9c2cb8c634348b03ffbb700d6612ca653577e674
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

train_dir = '/content/CUB-200-2011-dataset/train'
val_dir = '/content/CUB-200-2011-dataset/test'

# Define transformations for data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

# Load the CUB dataset
image_datasets = {x: datasets.ImageFolder(root=train_dir if x == 'train' else val_dir, transform=data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2) for x in ['train', 'val']}

# Load pre-trained EfficientNet (e.g., EfficientNet-B0)
efficientnet = EfficientNet.from_pretrained('efficientnet-b0')

# Modify the classifier
num_features = efficientnet._fc.in_features
efficientnet._fc = nn.Linear(num_features, 200)  # Set to 200 for CUB dataset

# Move model to device
efficientnet = efficientnet.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(efficientnet.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 25
for epoch in range(num_epochs):
    efficientnet.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in dataloaders['train']:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = efficientnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(dataloaders['train'].dataset)
    train_accuracy = correct_train / total_train

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Accuracy: {train_accuracy:.2%}")

    # Evaluation loop
    efficientnet.eval()
    correct_val = 0
    total_val = 0
    val_running_loss = 0.0

    with torch.no_grad():
        for images, labels in dataloaders['val']:
            images, labels = images.to(device), labels.to(device)
            outputs = efficientnet(images)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            val_loss = criterion(outputs, labels)
            val_running_loss += val_loss.item() * images.size(0)

    val_epoch_loss = val_running_loss / len(dataloaders['val'].dataset)
    val_accuracy = correct_val / total_val

    print(f"Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}")

# Print final test accuracy
print(f"Final Test Accuracy: {val_accuracy:.2%}")
